In [1]:
%pip install duckdb numpy


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlite3
import duckdb
sqlite_path = "./bpic2017.sqlite"
duckdb_path = "./bpic2017.duckdb"
query_names = {"sqlite": "sql.txt", "duckdb": "sql.txt"}
result_names = {"sqlite": "sqlite-res.json", "duckdb": "duckdb-res.json"}
queries = ["Q1","Q2","Q3","Q4","Q5","Q6","Q7"]
N = 10
def load_db(db):
    if db == "sqlite":
        source = sqlite3.connect(sqlite_path)
        # return source
        target = sqlite3.connect(":memory:")
        source.backup(target)
        return target
    if db == "duckdb":
        return duckdb.connect(duckdb_path)
def run_query(con,sql,db):
    if db == "sqlite":
        return con.execute(sql).fetchall()
    if db == "duckdb":
        return con.execute(sql).fetchall()

In [3]:
import time
import numpy as np
import json
for db in ["sqlite","duckdb"]:
    count = None
    durations = []
    for q in queries:
        if db == "sqlite" and q == "Q6": #Timeout!
            continue
        with open(f"{q}/{query_names[db]}") as f:
            sql = f.read()
            for i in range(0,N):
                con = load_db(db)
                start = time.time()
                qres = run_query(con,sql,db)
                duration = time.time() - start
                con.close()
                count = len(qres)
                durations.append(duration)
            with open(f"{q}/{result_names[db]}","w") as f:
                json.dump(durations,f)
            print(f"{db} for {q} counted {count} and took {1000*np.mean(durations)}")


sqlite for Q1 counted 31509 and took 328.46102714538574
sqlite for Q2 counted 42995 and took 507.8486442565918
sqlite for Q3 counted 23305 and took 360.14030774434406
sqlite for Q4 counted 31509 and took 671.8177020549774
sqlite for Q5 counted 31509 and took 677.9079294204712
sqlite for Q7 counted 74771 and took 860.1482351620991
duckdb for Q1 counted 31509 and took 46.17507457733154
duckdb for Q2 counted 42995 and took 73.35022687911987
duckdb for Q3 counted 23305 and took 66.95912679036459
duckdb for Q4 counted 31509 and took 81.05118870735168
duckdb for Q5 counted 31509 and took 138.81301403045654
duckdb for Q6 counted 1 and took 130.39328257242838
duckdb for Q7 counted 74771 and took 152.24860055106026
